In [1]:
%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
import lightgbm as lgb
from metric import metrics_stat

In [2]:
train_dataset = MainDataset("dataset/train.csv")
test_dataset = MainDataset("dataset/test.csv", need_index = False)

/home/ivan/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
neighborhoods = Neighborhoods(train_dataset.index)
neighborhoods1 = Neighborhoods(train_dataset.index1)

In [4]:
# for i, o in enumerate(train_dataset.all_objects[:10000]):
#     if i % 1000 == 0:
#         print(i)
#     neighbor = neighborhoods.get_haversine_closest(o, 7)
#     for f_name, f in functions.items():
#         prices = [f(o, x[0], x[1]) for x in neighbor]
#         o.neighbors_scores[f_name] = np.mean(prices)

In [209]:
for prefix, neighborhoods_item in [("", neighborhoods)]:
    for i, o in enumerate(train_dataset.all_objects):
        o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_first = prices[0]
                price_min = np.min(prices)
                price_std = np.std(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
                price_std=0
                price_first=0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min
            o.neighbors_scores[prefix + f_name+"_std"] = price_std

253000
254000
255000
256000


In [210]:
for prefix, neighborhoods_item in [("", neighborhoods)]:
    for i, o in enumerate(test_dataset.all_objects):
        o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_first = prices[0]
                price_min = np.min(prices)
                price_std = np.std(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
                price_std=0
                price_first=0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min
            o.neighbors_scores[prefix + f_name+"_std"] = price_std
            o.neighbors_scores[prefix + f_name+"_first"] = price_first

0
1000
2000


In [496]:
for prefix, neighborhoods_item in [("", neighborhoods)]:
    for i, o in enumerate(train_dataset.all_objects):
        o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 2)
        for i in range(1):
            n = neighbor[i]
            o.neighbors_scores["dist_" + str(i)] = n[1]
            o.neighbors_scores["total_price_" + str(i)] = n[0].row["per_square_meter_price"]* n[0].row["total_square"]
            o.neighbors_scores["square_price_" + str(i)] = n[0].row["per_square_meter_price"]


2
1
1
2
1
2
1
1
1
2
1
5
1
1
1
2
3
1
1
1
2
2
1
1
1
1
1
2
1
5
2
1
2
2
1
1
5
1
1
1
3
1
1
3
1
1
1
1
2
2
1
1
4
1
1
1
1
1
1
1
1
1
1
3
1
4
1
1
1
1
1
1
2
1
1
1
3
1
2
1
2
3
1
2
1
3
1
3
2
1
1
1
5
1
4
1
1
1
6
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
3
1
1
1
1
1
2
1
3
1
1
1
1
6
1
6
2
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
4
1
8
1
1
2
1
3
3
1
2
1
5
2
1
2
1
4
1
2
1
1
1
4
2
3
1
1
2
1
1
1
2
1
1
1
2
1
1
1
4
1
1
2
4
1
1
1
3
1
2
2
1
1
3
5
1
2
11
1
3
2
3
1
1
1
1
2
1
1
1
1
1
1
2
1
3
1
1
6
2
2
1
1
3
1
3
1
1
1
1
2
1
1
1
1
6
5
1
5
1
1
4
1
1
3
1
5
5
2
3
1
1
2
1
1
1
3
3
1
1
1
7
2
2
1
1
3
1
2
1
1
1
2
1
2
1
1
1
1
1
3
3
1
2
3
3
1
1
11
1
3
2
1
3
3
1
6
2
1
2
1
1
1
1
1
1
2
1
1
3
4
1
1
2
1
1
2
1
2
1
1
7
1
1
1
1
1
2
1
1
2
4
1
2
1
1
1
1
1
2
4
7
2
1
1
1
2
1
3
1
3
2
3
3
1
2
1
1
1
1
1
1
2
2
1
2
6
4
1
2
4
1
1
8
1
1
1
1
1
1
2
1
2
3
2
2
1
2
1
1
3
2
2
1
1
1
1
1
2
2
1
1
2
7
1
1
1
2
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
2
2
1
1
1
7
1
1
1
1
4
8
1
1
1
4
2
1
1
3
1
2
1
1
1
4
1
1
1
1
1
2
1
1
2
1
3
2
1
1
1
1
1
1
1
1
1
3
5
2
1
9
1
2
4


2
9
1
1
1
5
5
5
5
5
5
5
5
1
1
1
1
2
1
1
9
1
4
4
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
2
2
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
3
1
7
4
1
3
2
2
1
1
1
1
4
4
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
3
1
1
1
3
3
2
3
2
1
1
3
1
2
1
1
1
1
1
5
2
1
4
2
2
1
2
1
1
1
7
1
2
3
1
3
1
1
2
1
1
1
1
2
1
2
1
1
1
2
3
1
1
1
1
1
2
3
3
1
1
1
1
1
5
1
1
1
1
1
1
1
1
1
2
2
1
2
1
2
4
22
1
1
1
1
1
1
1
3
1
2
2
3
2
1
1
3
2
1
1
2
4
2
18
1
1
3
2
8
1
2
1
1
2
4
1
1
2
2
2
1
1
1
1
1
1
2
2
1
1
1
2
1
1
1
1
1
1
2
1
1
3
1
2
1
1
2
2
1
5
2
1
2
1
1
2
1
1
1
2
1
1
1
2
2
2
2
2
4
2
1
3
1
1
1
1
1
2
2
1
2
1
1
1
5
2
5
7
1
1
3
1
7
2
1
1
4
1
2
1
1
1
1
1
1
4
1
2
6
1


In [498]:


raw_dense_features = [
 'lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
       'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
       'osm_historic_points_in_0.005', 'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01', 
       'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square'
       ]

raw_cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']



good_f = ["F_total_price_std", "F_simple_price_median", "F_simple_price_max",
          "F_square_frac_True_median", "F_square_True_median",
         "F_total_price_max"
         ]

good_f = []
for i in range(1):
    good_f.append("dist_" + str(i))
    good_f.append("total_price_" + str(i))
    good_f.append("square_price_" + str(i))

def getX(o):
    x = []
    for f_name, score in o.neighbors_scores.items():
#         if f_name  in good_f:
        x.append(score)
     
    for n in raw_dense_features:
        x.append(o.row[n])
    for n in raw_cat_features:
        x.append(o.row[n])
    return pd.Series(x)


In [499]:
columns = []
# for p in ["",
# #           "P1_"
#          ]:
#     for t in ["_median", "_min", "_max", "_std"]:
#         columns.extend([p + f_name + t for f_name, f in functions.functions.items() ])
columns.extend(good_f)
columns = [c for c in columns if c]
    # columns = []
columns.extend(raw_dense_features)
columns.extend(raw_cat_features)

In [500]:
# X0 = [getX(o) for o in train_dataset.all_objects[:1] if o.row["price_type"] == 0]
# X0 = pd.concat(X0, axis=1).T
# X0.columns=columns
# Y0 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 0]
# X0["floor"] = X0["floor"].astype("str")

In [501]:
X1 = [getX(o) for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = pd.concat(X1, axis=1).T
X1.columns=columns
Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = X1.fillna(-999)


In [511]:
floor_mapping = {"Цоколь": -1, "цокольный":-1, "Подвал": -1,"подвал": -1,"цоколь": -1,
                 "-1.0": -1,"0.0": -1,"-2.0": -1, "Техническое подполье":-1,
                   
                 "1 этаж": 1,"1.0": 1,  "1-й":1, "фактически на уровне 1 этажа":1, "1, антресоль":1,
                 "2.0": 2, "2-й":2,
                 "3.0": 3, "3 этаж":3,
                   "4.0":4, "4 этаж":4,
                  "5.0": 5,
                 "6.0": 6,
                 "7.0": 7,
                 "8.0": 8,
                 "9.0": 9,
                 "10.0": 10,
                 
                 "1,2":1,"1, 2":1,"1,2,3,4":1, "1, 2, 3":1, "1,2,3,4,5":1, "1-й, 2-й":1, "1, 2 этаж":1,
                 "подвал, 1, 2, 3":1, "подвал, 1":1,  "1,2,подвал":1,
                 "1,2,3, антресоль, технический этаж":1, "1 (по док-м цоколь)":1,
                 "цокольный, 1,2":1,"1-7":8,
                 
                 "1, подвал":0,  "1, цоколь":0, 
                  "цоколь, 1":0, "подвал, 1 и 4 этаж":0,
                 
                  "подвал, 3. 4 этаж":3,
                 "29":100, "36":100
                }

floor_count_mapping = {
    "1, подвал":2,
     "1,2":2,
    "1, 2":2,
    "1,2,3,4":2,
    "цоколь, 1, 2,3,4,5,6":7,
    "1, 2, 3":3,
    "цоколь, 1":2,
    "подвал , 1":2,
    
    "1, цоколь":2,
    "Цоколь, 1,2(мансарда)":3,
    "подвал,1,2,3,4,5":6,
    "1-3 этажи":3,
    "1 этаж, подвал":2,
    "1,2,3,4, подвал":5,
    "подвал,1,2,3":4,
    "подвал , 1":2,
    "1,2,3 этаж, подвал":4,
    "1,2,3,4,5":5,
    "2, 3, 4, тех.этаж":4,
    "технический этаж,5,6":3,
    " 1-2, подвальный":3,
    "1, 2, 3, мансардный":4,
    "1,2":2,
    "подвал, 1 и 4 этаж":3,
    "4, 5":2,
    "1 + подвал (без отделки)":2,
    "подвал, 1,2,3":4,
    "подвал, 1-4 этаж":5,
    
    "1,2,антресоль":2,
    " 1, 2, Антресоль":2,
    "подва, 1.2 этаж":3,
    "подвал, 1-7, техэтаж":8,
    "1,2,3":3,
    "3,4":2,
    "1-й, подвал":2,
    "1-й, 2-й":2,
    "1,2,подвал":3,
    "1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)":3,
    "1, подвал, антресоль":2,
    "1.2":2,
    "1-3":3,
    "1,2,подвал":3,
    "подвал , 1":2,
    "подвал, 3. 4 этаж":3,
    "1, 2 этаж":2,
    "подвал, 1, 2, 3":3,
    "подвал, 1":2,
    "1,2,подвал":3,
    "1,2,3, антресоль, технический этаж":4,
    "цокольный, 1,2":3,
    "1-7":8,
    
    
    
    
}

has_basement = [
    "подвал","цоколь","1, подвал","цокольный","Техническое подполье","цоколь, 1","1, цоколь","1 этаж, подвал","подвал , 1","Цоколь, 1,2(мансарда)",
     "1,2,3 этаж, подвал","цоколь, 1, 2,3,4,5,6","подвал, 1","1-й, подвал","1 (по док-м цоколь)","подвал, 2","подвал, 1 и 4 этаж","2, 3, 4, тех.этаж",
     "1,2,3,4, подвал","подвал,1,2,3,4,5","1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)","технический этаж,5,6","подвал,1,2,3","подва, 1.2 этаж",
     "подвал, цоколь, 1 этаж","подвал, 1, 2","подвал, 1-4 этаж","1, подвал, антресоль","1 + подвал (без отделки)"," 1-2, подвальный","подвал, 3. 4 этаж",
     "подвал, 1,2,3","тех.этаж (6)","цокольный, 1,2","1,2,3, антресоль, технический этаж","Цоколь","1,2,подвал","подвал, 1, 2, 3","1 (Цокольный этаж)","подвал, 1-3","подвал, 1-7, техэтаж"
]

X1["has_basement"]=0
X1.loc[ X1["floor"].isna(), "has_basement"] = -999

for k in has_basement:
    X1.loc[  X1["floor"] == k, "has_basement"] = 1
    
    
X1["floor_count"]=1
X1.loc[  X1["floor"].isna(), "floor_count"] = -999
for k,v in floor_count_mapping.items():
    X1.loc[  X1["floor"] == k, "floor_count"] = v

for k,v in floor_mapping.items():
    X1.loc[  X1["floor"] == k, "floor"] = v
X1["floor"]=X1["floor"].fillna(-999)
X1["floor"]=X1.floor.astype("str")



In [503]:
# Xtest1 = [getX(o) for o in test_dataset.all_objects]
# Xtest1 = pd.concat(Xtest1, axis=1).T
# Xtest1.columns=columns
# Xtest1 = Xtest1.fillna(-999)

In [504]:
# X0 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 0])
# X1 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 1])
# Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]

In [505]:
# np.save("x0", X0)
# np.save("y0", Y0)
# np.save("x1", X1)
# np.save("y1", Y1)

In [512]:
# X0 = X0.fillna(-999)
X_train, X_val, Y_train, Y_val = train_test_split(X1, Y1, test_size=0.1, random_state=46)

In [513]:
train_data = Pool(data=X_train,
                  cat_features=raw_cat_features,
                  label=np.log(np.array(Y_train)))
#                   label=np.array(Y_train))


val_data = Pool(data=X_val,
                cat_features=raw_cat_features,
#                   label=np.array(Y_val))
                  label=np.log(np.array(Y_val)))

In [514]:
cat_model = CatBoostRegressor(
    l2_leaf_reg=6,
    bagging_temperature=1.5,
#     random_strength=2,
    learning_rate=0.014,
    iterations=8000,
    eval_metric="RMSE",
)
cat_model.fit(train_data,eval_set=val_data, use_best_model=False)



0:	learn: 0.5861157	test: 0.5695140	best: 0.5695140 (0)	total: 6.37ms	remaining: 1m 3s
1:	learn: 0.5834318	test: 0.5667465	best: 0.5667465 (1)	total: 11.9ms	remaining: 59.4s
2:	learn: 0.5809958	test: 0.5642664	best: 0.5642664 (2)	total: 17.2ms	remaining: 57.3s
3:	learn: 0.5782484	test: 0.5615166	best: 0.5615166 (3)	total: 23ms	remaining: 57.5s
4:	learn: 0.5758041	test: 0.5589144	best: 0.5589144 (4)	total: 27.6ms	remaining: 55.2s
5:	learn: 0.5730918	test: 0.5561514	best: 0.5561514 (5)	total: 32.4ms	remaining: 53.9s
6:	learn: 0.5705621	test: 0.5535265	best: 0.5535265 (6)	total: 37.5ms	remaining: 53.6s
7:	learn: 0.5680900	test: 0.5509569	best: 0.5509569 (7)	total: 43.5ms	remaining: 54.3s
8:	learn: 0.5657200	test: 0.5483891	best: 0.5483891 (8)	total: 48ms	remaining: 53.2s
9:	learn: 0.5633950	test: 0.5458729	best: 0.5458729 (9)	total: 53.2ms	remaining: 53.1s
10:	learn: 0.5609484	test: 0.5432463	best: 0.5432463 (10)	total: 57.9ms	remaining: 52.6s
11:	learn: 0.5587139	test: 0.5408474	best: 0.

113:	learn: 0.4183502	test: 0.3972433	best: 0.3972433 (113)	total: 601ms	remaining: 52.2s
114:	learn: 0.4176099	test: 0.3965281	best: 0.3965281 (114)	total: 608ms	remaining: 52.3s
115:	learn: 0.4169125	test: 0.3959241	best: 0.3959241 (115)	total: 615ms	remaining: 52.4s
116:	learn: 0.4162384	test: 0.3952756	best: 0.3952756 (116)	total: 621ms	remaining: 52.4s
117:	learn: 0.4156236	test: 0.3946740	best: 0.3946740 (117)	total: 626ms	remaining: 52.4s
118:	learn: 0.4150411	test: 0.3940671	best: 0.3940671 (118)	total: 631ms	remaining: 52.4s
119:	learn: 0.4143610	test: 0.3933532	best: 0.3933532 (119)	total: 637ms	remaining: 52.5s
120:	learn: 0.4136521	test: 0.3926064	best: 0.3926064 (120)	total: 646ms	remaining: 52.7s
121:	learn: 0.4130242	test: 0.3919356	best: 0.3919356 (121)	total: 651ms	remaining: 52.7s
122:	learn: 0.4124964	test: 0.3914031	best: 0.3914031 (122)	total: 656ms	remaining: 52.7s
123:	learn: 0.4118723	test: 0.3907739	best: 0.3907739 (123)	total: 662ms	remaining: 52.7s
124:	learn

220:	learn: 0.3707381	test: 0.3522545	best: 0.3522545 (220)	total: 1.21s	remaining: 53.7s
221:	learn: 0.3704515	test: 0.3520942	best: 0.3520942 (221)	total: 1.22s	remaining: 53.8s
222:	learn: 0.3701943	test: 0.3519284	best: 0.3519284 (222)	total: 1.23s	remaining: 53.8s
223:	learn: 0.3699495	test: 0.3517250	best: 0.3517250 (223)	total: 1.23s	remaining: 53.8s
224:	learn: 0.3696678	test: 0.3514745	best: 0.3514745 (224)	total: 1.24s	remaining: 53.8s
225:	learn: 0.3694196	test: 0.3512236	best: 0.3512236 (225)	total: 1.24s	remaining: 53.8s
226:	learn: 0.3692106	test: 0.3510739	best: 0.3510739 (226)	total: 1.25s	remaining: 53.7s
227:	learn: 0.3689892	test: 0.3508868	best: 0.3508868 (227)	total: 1.25s	remaining: 53.7s
228:	learn: 0.3687637	test: 0.3506934	best: 0.3506934 (228)	total: 1.26s	remaining: 53.7s
229:	learn: 0.3685210	test: 0.3504654	best: 0.3504654 (229)	total: 1.26s	remaining: 53.7s
230:	learn: 0.3682484	test: 0.3502130	best: 0.3502130 (230)	total: 1.27s	remaining: 53.7s
231:	learn

329:	learn: 0.3495838	test: 0.3358180	best: 0.3358180 (329)	total: 1.82s	remaining: 53.2s
330:	learn: 0.3494046	test: 0.3356008	best: 0.3356008 (330)	total: 1.82s	remaining: 53.2s
331:	learn: 0.3492490	test: 0.3354673	best: 0.3354673 (331)	total: 1.83s	remaining: 53.2s
332:	learn: 0.3490949	test: 0.3353317	best: 0.3353317 (332)	total: 1.83s	remaining: 53.2s
333:	learn: 0.3489191	test: 0.3352048	best: 0.3352048 (333)	total: 1.84s	remaining: 53.2s
334:	learn: 0.3488039	test: 0.3351222	best: 0.3351222 (334)	total: 1.84s	remaining: 53.2s
335:	learn: 0.3486811	test: 0.3350298	best: 0.3350298 (335)	total: 1.85s	remaining: 53.2s
336:	learn: 0.3485749	test: 0.3349105	best: 0.3349105 (336)	total: 1.85s	remaining: 53.1s
337:	learn: 0.3484131	test: 0.3347755	best: 0.3347755 (337)	total: 1.86s	remaining: 53.2s
338:	learn: 0.3483001	test: 0.3346645	best: 0.3346645 (338)	total: 1.86s	remaining: 53.1s
339:	learn: 0.3481967	test: 0.3346128	best: 0.3346128 (339)	total: 1.87s	remaining: 53.1s
340:	learn

448:	learn: 0.3359321	test: 0.3266319	best: 0.3266319 (448)	total: 2.42s	remaining: 51.6s
449:	learn: 0.3357909	test: 0.3265940	best: 0.3265940 (449)	total: 2.43s	remaining: 51.6s
450:	learn: 0.3356995	test: 0.3265284	best: 0.3265284 (450)	total: 2.44s	remaining: 51.6s
451:	learn: 0.3356248	test: 0.3264782	best: 0.3264782 (451)	total: 2.44s	remaining: 51.6s
452:	learn: 0.3355673	test: 0.3264355	best: 0.3264355 (452)	total: 2.45s	remaining: 51.6s
453:	learn: 0.3354558	test: 0.3263844	best: 0.3263844 (453)	total: 2.45s	remaining: 51.6s
454:	learn: 0.3353622	test: 0.3263811	best: 0.3263811 (454)	total: 2.46s	remaining: 51.7s
455:	learn: 0.3352314	test: 0.3263272	best: 0.3263272 (455)	total: 2.47s	remaining: 51.7s
456:	learn: 0.3351997	test: 0.3263073	best: 0.3263073 (456)	total: 2.47s	remaining: 51.6s
457:	learn: 0.3351037	test: 0.3262568	best: 0.3262568 (457)	total: 2.48s	remaining: 51.6s
458:	learn: 0.3350159	test: 0.3261986	best: 0.3261986 (458)	total: 2.48s	remaining: 51.6s
459:	learn

...

9525:	learn: 0.1480199	test: 0.3480627	best: 0.3184523 (935)	total: 51.8s	remaining: 2.58s
9526:	learn: 0.1480194	test: 0.3480626	best: 0.3184523 (935)	total: 51.8s	remaining: 2.57s
9527:	learn: 0.1480134	test: 0.3480612	best: 0.3184523 (935)	total: 51.8s	remaining: 2.57s
9528:	learn: 0.1479987	test: 0.3480629	best: 0.3184523 (935)	total: 51.8s	remaining: 2.56s
9529:	learn: 0.1479977	test: 0.3480613	best: 0.3184523 (935)	total: 51.8s	remaining: 2.56s
9530:	learn: 0.1479960	test: 0.3480647	best: 0.3184523 (935)	total: 51.8s	remaining: 2.55s
9531:	learn: 0.1479946	test: 0.3480658	best: 0.3184523 (935)	total: 51.9s	remaining: 2.54s
9532:	learn: 0.1479885	test: 0.3480662	best: 0.3184523 (935)	total: 51.9s	remaining: 2.54s
9533:	learn: 0.1479885	test: 0.3480651	best: 0.3184523 (935)	total: 51.9s	remaining: 2.54s
9534:	learn: 0.1479842	test: 0.3480617	best: 0.3184523 (935)	total: 51.9s	remaining: 2.53s
9535:	learn: 0.1479839	test: 0.3480607	best: 0.3184523 (935)	total: 51.9s	remaining: 2.52s

9632:	learn: 0.1470721	test: 0.3482467	best: 0.3184523 (935)	total: 52.4s	remaining: 2s
9633:	learn: 0.1470718	test: 0.3482459	best: 0.3184523 (935)	total: 52.4s	remaining: 1.99s
9634:	learn: 0.1470620	test: 0.3482494	best: 0.3184523 (935)	total: 52.4s	remaining: 1.99s
9635:	learn: 0.1470504	test: 0.3482455	best: 0.3184523 (935)	total: 52.4s	remaining: 1.98s
9636:	learn: 0.1470486	test: 0.3482446	best: 0.3184523 (935)	total: 52.4s	remaining: 1.98s
9637:	learn: 0.1470298	test: 0.3482563	best: 0.3184523 (935)	total: 52.4s	remaining: 1.97s
9638:	learn: 0.1470139	test: 0.3482635	best: 0.3184523 (935)	total: 52.5s	remaining: 1.96s
9639:	learn: 0.1470118	test: 0.3482658	best: 0.3184523 (935)	total: 52.5s	remaining: 1.96s
9640:	learn: 0.1470092	test: 0.3482669	best: 0.3184523 (935)	total: 52.5s	remaining: 1.95s
9641:	learn: 0.1470006	test: 0.3482680	best: 0.3184523 (935)	total: 52.5s	remaining: 1.95s
9642:	learn: 0.1469902	test: 0.3482598	best: 0.3184523 (935)	total: 52.5s	remaining: 1.94s
96

9741:	learn: 0.1460503	test: 0.3484409	best: 0.3184523 (935)	total: 53s	remaining: 1.4s
9742:	learn: 0.1460336	test: 0.3484498	best: 0.3184523 (935)	total: 53s	remaining: 1.4s
9743:	learn: 0.1460124	test: 0.3484438	best: 0.3184523 (935)	total: 53s	remaining: 1.39s
9744:	learn: 0.1460089	test: 0.3484418	best: 0.3184523 (935)	total: 53s	remaining: 1.39s
9745:	learn: 0.1459977	test: 0.3484509	best: 0.3184523 (935)	total: 53s	remaining: 1.38s
9746:	learn: 0.1459870	test: 0.3484602	best: 0.3184523 (935)	total: 53.1s	remaining: 1.38s
9747:	learn: 0.1459811	test: 0.3484550	best: 0.3184523 (935)	total: 53.1s	remaining: 1.37s
9748:	learn: 0.1459714	test: 0.3484583	best: 0.3184523 (935)	total: 53.1s	remaining: 1.37s
9749:	learn: 0.1459591	test: 0.3484707	best: 0.3184523 (935)	total: 53.1s	remaining: 1.36s
9750:	learn: 0.1459572	test: 0.3484710	best: 0.3184523 (935)	total: 53.1s	remaining: 1.35s
9751:	learn: 0.1459436	test: 0.3484684	best: 0.3184523 (935)	total: 53.1s	remaining: 1.35s
9752:	learn

9849:	learn: 0.1450014	test: 0.3486945	best: 0.3184523 (935)	total: 53.6s	remaining: 816ms
9850:	learn: 0.1449828	test: 0.3486923	best: 0.3184523 (935)	total: 53.6s	remaining: 811ms
9851:	learn: 0.1449777	test: 0.3486932	best: 0.3184523 (935)	total: 53.6s	remaining: 806ms
9852:	learn: 0.1449774	test: 0.3486932	best: 0.3184523 (935)	total: 53.6s	remaining: 800ms
9853:	learn: 0.1449773	test: 0.3486922	best: 0.3184523 (935)	total: 53.6s	remaining: 795ms
9854:	learn: 0.1449761	test: 0.3487109	best: 0.3184523 (935)	total: 53.6s	remaining: 789ms
9855:	learn: 0.1449637	test: 0.3487186	best: 0.3184523 (935)	total: 53.6s	remaining: 784ms
9856:	learn: 0.1449490	test: 0.3487251	best: 0.3184523 (935)	total: 53.7s	remaining: 778ms
9857:	learn: 0.1449437	test: 0.3487213	best: 0.3184523 (935)	total: 53.7s	remaining: 773ms
9858:	learn: 0.1449378	test: 0.3487274	best: 0.3184523 (935)	total: 53.7s	remaining: 768ms
9859:	learn: 0.1449342	test: 0.3487315	best: 0.3184523 (935)	total: 53.7s	remaining: 762ms

9956:	learn: 0.1441639	test: 0.3487519	best: 0.3184523 (935)	total: 54.2s	remaining: 234ms
9957:	learn: 0.1441517	test: 0.3487560	best: 0.3184523 (935)	total: 54.2s	remaining: 229ms
9958:	learn: 0.1441420	test: 0.3487588	best: 0.3184523 (935)	total: 54.2s	remaining: 223ms
9959:	learn: 0.1441417	test: 0.3487590	best: 0.3184523 (935)	total: 54.2s	remaining: 218ms
9960:	learn: 0.1441367	test: 0.3487700	best: 0.3184523 (935)	total: 54.2s	remaining: 212ms
9961:	learn: 0.1441301	test: 0.3487693	best: 0.3184523 (935)	total: 54.2s	remaining: 207ms
9962:	learn: 0.1441080	test: 0.3487738	best: 0.3184523 (935)	total: 54.2s	remaining: 201ms
9963:	learn: 0.1440825	test: 0.3487797	best: 0.3184523 (935)	total: 54.3s	remaining: 196ms
9964:	learn: 0.1440690	test: 0.3487744	best: 0.3184523 (935)	total: 54.3s	remaining: 191ms
9965:	learn: 0.1440541	test: 0.3487535	best: 0.3184523 (935)	total: 54.3s	remaining: 185ms
9966:	learn: 0.1440393	test: 0.3487499	best: 0.3184523 (935)	total: 54.3s	remaining: 180ms

In [515]:
cat_preds = cat_model.predict(X_val, ntree_end=10000)
metrics_stat(np.array(Y_val), np.exp(np.array(cat_preds))/1.15)
# metrics_stat(np.array(Y_val), np.array(cat_preds)/1.15)

{'mape': 0.24982254427107736,
 'mdape': 0.1984815569321648,
 'rmse': 42304.11365891082,
 'r2': 0.6434210510876864,
 'raif_metric': 1.3631037196759865}

In [ ]:
# {'mape': 0.24475652382438295,
#  'mdape': 0.20151566000168558,
#  'rmse': 42115.51144634587,
#  'r2': 0.6465933982105105,
#  'raif_metric': 1.319543781125282}

In [339]:
# {'mape': 0.21144235856299823,
#  'mdape': 0.1852917036522351,
#  'rmse': 45127.9217242655,
#  'r2': 0.7148011343698422,
#  'raif_metric': 0.975592417222578}

In [516]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)

,Feature Id,Importances
0,total_square,12.848621
1,city,6.517691
2,region,6.320359
3,square_price_0,4.485699
4,osm_city_nearest_name,4.029269
5,osm_catering_points_in_0.01,2.533008
6,total_price_0,2.517001
7,floor,2.513952
8,osm_crossing_closest_dist,2.283963
9,dist_0,2.231476


In [303]:
# pd.set_option("display.max_rows", None) 
# cat_model.get_feature_importance(data=val_data,prettified=True)


,Feature Id,Importances
0,city,6.434054
1,total_square,6.255149
2,region,5.065453
3,osm_city_nearest_name,3.861004
4,floor,2.824273
5,osm_crossing_closest_dist,2.359783
6,realty_type,1.960673
7,osm_transport_stop_closest_dist,1.907761
8,lng,1.838836
9,osm_subway_closest_dist,1.711441


In [ ]:
{'mape': 0.21248556703899782,
 'mdape': 0.17348966518138254,
 'rmse': 44143.024518500744,
 'r2': 0.7271139725101594,
 'raif_metric': 1.0252440567552334}

In [171]:
for c in raw_cat_features:
    X_train[c] = pd.Categorical(X_train[c])
    X_val[c] = pd.Categorical(X_val[c])

/home/ivan/projects/raif/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ivan/projects/raif/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [172]:
lgb_train = lgb.Dataset(X_train, np.log(np.array(Y_train)), categorical_feature=raw_cat_features)
lgb_eval = lgb.Dataset(X_val, np.log(np.array(Y_val)), reference=lgb_train, categorical_feature=raw_cat_features)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
#     'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.03,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'verbose': 0
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=400,
                valid_sets=lgb_eval,
#                 early_stopping_rounds=5
               )



/home/ivan/projects/raif/venv/lib/python3.6/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/ivan/projects/raif/venv/lib/python3.6/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/ivan/projects/raif/venv/lib/python3.6/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001342 seconds.
You can set `force_col_wise=true` to remove the overhead.
[1]	valid_0's l2: 0.357594
[2]	valid_0's l2: 0.343677
[3]	valid_0's l2: 0.330922
[4]	valid_0's l2: 0.319137
[5]	valid_0's l2: 0.307408
[6]	valid_0's l2: 0.296478
[7]	valid_0's l2: 0.2864
[8]	valid_0's l2: 0.276528
[9]	valid_0's l2: 0.267362
[10]	valid_0's l2: 0.258655
[11]	valid_0's l2: 0.250234
[12]	valid_0's l2: 0.242506
[13]	valid_0's l2: 0.235433
[14]	valid_0's l2: 0.228098
[15]	valid_0's l2: 0.221592
[16]	valid_0's l2: 0.215274
[17]	valid_0's l2: 0.208903
[18]	valid_0's l2: 0.203138
[19]	valid_0's l2: 0.198341
[20]	valid_0's l2: 0.19295
[21]	valid_0's l2: 0.187799
[22]	valid_0's l2: 0.18321
[23]	valid_0's l2: 0.178666
[24]	valid_0's l2: 0.174677
[25]	valid_0's l2: 0.170894
[26]	valid_0's l2: 0.167451
[27]	valid_0's l2: 0.16428
[28]	valid_0's l2: 0.161052
[29]	valid_0's l2: 0.157948
[30]	valid_0's l2: 0.154931
[31]	vali

[300]	valid_0's l2: 0.0938351
[301]	valid_0's l2: 0.0939107
[302]	valid_0's l2: 0.0939615
[303]	valid_0's l2: 0.0939337
[304]	valid_0's l2: 0.0939071
[305]	valid_0's l2: 0.0939377
[306]	valid_0's l2: 0.0939549
[307]	valid_0's l2: 0.0938884
[308]	valid_0's l2: 0.0939688
[309]	valid_0's l2: 0.0939977
[310]	valid_0's l2: 0.0940337
[311]	valid_0's l2: 0.094078
[312]	valid_0's l2: 0.0940609
[313]	valid_0's l2: 0.09402
[314]	valid_0's l2: 0.0940286
[315]	valid_0's l2: 0.0940259
[316]	valid_0's l2: 0.0940754
[317]	valid_0's l2: 0.0940382
[318]	valid_0's l2: 0.0940186
[319]	valid_0's l2: 0.0940008
[320]	valid_0's l2: 0.0940269
[321]	valid_0's l2: 0.094062
[322]	valid_0's l2: 0.0939401
[323]	valid_0's l2: 0.0939697
[324]	valid_0's l2: 0.0940321
[325]	valid_0's l2: 0.0940395
[326]	valid_0's l2: 0.0939835
[327]	valid_0's l2: 0.0940863
[328]	valid_0's l2: 0.0940568
[329]	valid_0's l2: 0.0940856
[330]	valid_0's l2: 0.0940838
[331]	valid_0's l2: 0.0940532
[332]	valid_0's l2: 0.0939998
[333]	valid_0'

In [173]:
gbm_preds = gbm.predict(X_val)
metrics_stat(np.array(Y_val), np.exp(np.array(gbm_preds))/1.15)

{'mape': 0.2246983209946315,
 'mdape': 0.19286489919725555,
 'rmse': 86287.28295616168,
 'r2': 0.348793958059304,
 'raif_metric': 1.0668941017647746}

In [178]:
gbm_preds = gbm.predict(X_val)
metrics_stat(np.array(Y_val), np.exp(np.array(gbm_preds+cat_preds)/2)/1.15)

{'mape': 0.22441295764167724,
 'mdape': 0.18608382531036846,
 'rmse': 86276.38103433553,
 'r2': 0.34895850024058317,
 'raif_metric': 1.0176804758362274}

In [59]:
Xtest1 = Xtest1.fillna(-999)
test_data = Pool(data=Xtest1,
                  cat_features=raw_cat_features)
price0_preds =  np.exp(cat_model.predict(test_data))/1.15

pd.DataFrame({"id": [o.row['id'] for o in test_dataset.all_objects],
             "per_square_meter_price": price0_preds
             }).to_csv("log2.csv", index=False)